In [1]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
import nltk
import warnings
import urllib.request
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from transformers import BertTokenizer, BertModel
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
warnings.filterwarnings('ignore')

# 데이터 라벨링

In [ ]:
# 일반대화 wav 데이터 불러오기 및 라벨링

os.chdir('/content/drive/MyDrive/1조 조별프로젝트/유튭 합친 json')
text_dir = '/content/drive/MyDrive/1조 조별프로젝트/유튭 합친 json/'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass


d = pd.DataFrame(d)
normal_dir = '/content/drive/MyDrive/1조 조별프로젝트/일반 보이스 합친 파일'
normal_files = os.listdir(normal_dir)
d['wav'] = normal_files
normal= d
normal.rename(columns={0:'txt'},inplace=True)
normal['voice_fishing']=0
normal

,txt,wav,voice_fishing
0,내가 나한테 잘해주면 돼요.혹시 여러분들 자식들가운데?지금 손자 손녀키운다고 먹고 ...,1-converted.ogg,0
1,운명을 바꿔 놓습니다.서울대 음대를 나왔는데요.서울대가 중요한 게 아닙니다.결국 이...,10-converted.ogg,0
2,"네, 감사합니다.네, 감사합니다.서울대 면접없는 판 모국입니다 검사대 여름말씀이신건...",100-converted.ogg,0
3,그러나 인류는 절대 내가 내 말에 절대 라는 용어를 여간 해서는 절대 안 된다는 얘...,11-converted.ogg,0
4,그런데 외국사람이결국은 집 이사 이제 세방을 얻어 는데 그때 50 전세 한 달에 5...,12-converted.ogg,0
...,...,...,...
95,회원 가입을 했거든요 하신다하는 건가요?혹시 방법을 높여야 하는지 잘 몰라서 보는데...,95-converted.ogg,0
96,근데 이제 장소가 이제 구애가 없다고 하시면 저희가 그을지로 고용노동부건물에 있는 ...,96-converted.ogg,0
97,감사합니다.확인 도와드릴 텐데 방송한 기다려 주시겠어요?여기야 기자 감사드리고요.거...,97-converted.ogg,0
98,네 감사합니다.저 인사담당자신청했는데요 그 a니까 아직도 않았어요?네 감사합니다.,98-converted.ogg,0


In [ ]:
# 보이스피싱 wav 데이터 불러오기 및 라벨링

os.chdir('/Users/medici/Downloads/보이스피싱 json')
text_dir = '/Users/medici/Downloads/보이스피싱 json/'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass
d = pd.DataFrame(d)
voice_dir = '/Users/medici/Downloads/보이스피싱 음성/'
voice_files = os.listdir(voice_dir)
d['wav'] = voice_files
voice= d
voice.rename(columns={0:'txt'},inplace=True)
voice['voice_fishing']=1

voice

,txt,wav,voice_fishing
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1-1-converted.ogg,1
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1-2-converted.ogg,1
2,그럼 최근 삼 년 사이에 본인개인정보가 담긴 물품 분실하신 적 전혀 없으십니까?제가...,1-3-converted.ogg,1
3,사실 있는 그대로만 진술하시면 되시는데 현재 계신 곳이 직장 이십니까.통합 불편하시...,1-4-converted.ogg,1
4,일단 이 두 통장은 범죄로 사용돼서 추가적인 피해자가 발생하지 않도록 본인 의사와 ...,1-5-converted.ogg,1
...,...,...,...
77,많은 분들이 연루돼 있는 부분들가운데서 저희가 지금 동시에 수사를 진행하다 보니까 ...,9-4.wav,1
78,이렇게 지금 전화 받아 가지고 있던 말아무도 본인이 지금 왜 보려는 전에 는지 본회...,9-5.wav,1
79,이제 최종 처리를 포함한 공범들하고는 이제 발견된 연관성이 없었기 때문에 일차적인 ...,9-6.wav,1
80,사건 으로 외모되셨으면 확인 한번 부탁드립니다.네 오세요.서울중앙지검에 김진수 아닙...,9-7.wav,1


In [ ]:
# 일반대화와 보이스피싱 합치기
df = pd.concat([voice,normal],axis=0)
df.reset_index(drop=True,inplace=True)

# 음성 데이터를 통한 보이스피싱 분류

In [ ]:
# wav 데이터 mfcc로 수치화

voice_spam_dir = '/Users/medici/Downloads/보이스피싱 음성/'
normal_dir = '/Users/medici/Downloads/일반대화 음성/'
df['filepath']=''
df['voice_feature']=''


for i in range(len(df)):
    if df.at[i,'voice_fishing']==1:
        df.at[i,'filepath'] = os.path.join(voice_spam_dir, df.at[i,'wav'])
        y, sr = librosa.load(df.at[i,'filepath'], sr=22050)
        y = librosa.util.normalize(y)  # 음량 조정
        mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_mean1 = np.mean(mfccs1, axis=1)
        df.at[i,'voice_feature'] = mfccs_mean1

    else:
        df.at[i,'filepath'] = os.path.join(normal_dir, df.at[i,'wav'])
        y, sr = librosa.load(df.at[i,'filepath'], sr=22050)
        y = librosa.util.normalize(y)  # 음량 조정
        mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_mean1 = np.mean(mfccs1, axis=1)
        df.at[i,'voice_feature'] = mfccs_mean1

In [ ]:
# 음성 데이터 학습

df_temp=df.drop(columns=['txt','wav','voice_fishing','filepath'])
X = df_temp
y = df['voice_fishing']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state= 46)
X_train1=list(X_train['voice_feature'])
X_test1=list(X_test['voice_feature'])
y_train1 = list(y_train)
y_test1 = list(y_test)

# 음성 데이터 학습

df_temp=df.drop(columns=['txt','wav','voice_fishing','filepath'])
X = df_temp
y = df['voice_fishing']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state= 18)
X_train1=list(X_train['voice_feature'])
X_test1=list(X_test['voice_feature'])
y_train1 = list(y_train)
y_test1 = list(y_test)


# randomforest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
parameters1 = {
    "n_estimators" : [10, 50, 1000, 2000],
    "max_features" : ["sqrt", "log2"],
    "max_depth" : [2,30, 50,70,100,150,200,300,400]
}
n_iter_search = 10
rf_rgs1 = RandomizedSearchCV(
    rf,
    param_distributions=parameters1,
    scoring="recall",
    n_jobs=-1,
    random_state=42,
    n_iter = n_iter_search
)
rf_rgs1.fit(X_train1, y_train1)
y_pred_rf1 = rf_rgs1.predict(X_test1)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test1, y_pred_rf1))
print("Recall:", recall_score(y_test1, y_pred_rf1))
print(' ')



# gradient boosting
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
parameters={
    'n_estimators': [1300,1500,1700],
    'max_depth': [15,20,22],
    'min_samples_leaf': [33,35,40],
    'min_samples_split': [500,700,900],
    'learning_rate': [0.2,0.1],
}
n_iter_search = 20
gb_kf_rgs = RandomizedSearchCV(
    gb,
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1,
    random_state=42,
    n_iter = n_iter_search
)
gb_kf_rgs.fit(X_train1, y_train1)
y_pred_gb1 = gb_kf_rgs.predict(X_test1)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test1, y_pred_gb1))
print("Recall:", recall_score(y_test1, y_pred_gb1))
print(' ')


# xgboost
xgb= XGBClassifier( n_estimators=100,n_jobs=-1)
parameters ={
     "n_estimators":[200,300,400],
     "learning_rate":[0.3, 0.5, 1.0],
     "max_depth" : [6,7],
     "gamma" : [0.1, 0.15],
     "subsample":[0.5, 0.6, 0.7],
     "colsample_bytree":[0.3, 0.5, 1],
}

xgb_gs =GridSearchCV(
    xgb,
    param_grid=parameters,
    scoring="recall",
    n_jobs=-1,
)
xgb_gs.fit(X_train1, y_train1)

y_pred_xgb1 = xgb_gs.predict(X_test1)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test1, y_pred_xgb1))
print("Recall:", recall_score(y_test1, y_pred_xgb1))
print(' ')

# Lightgbm
lgbm = LGBMClassifier( random_state=42, n_jobs=-1)

parameters ={
    "n_estimators":[100, 200, 300, 500, 1000],
    "learning_rate":[0.01, 0.1, 0.2, 0.5, 1],

    "max_depth" : [1,3,5,7],
    "min_split_gain" : [0, 0.1, 0.2, 0.4, 0.5],

    "subsample":[0.3, 0.5, 0.7, 0.9],
    "colsample_bytree":[0.3, 0.5, 0.7, 0.8, 0.9],

    "reg_alpha": [0, 0.01, 0.1, 0.5, 1, 10 ],
    "reg_lambda" : [0.01, 0.1, 0.5, 1, 10 ]
}
n_iter_search = 50
lgb_kf_rgs = RandomizedSearchCV(
    lgbm,
    param_distributions=parameters,
    scoring="recall",
    n_jobs=-1,
    random_state=42,
    n_iter = n_iter_search
)

lgb_kf_rgs.fit(X_train1, y_train1)
y_pred_lgbm = lgb_kf_rgs.predict(X_test1)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test1, y_pred_lgbm))
print("Recall:", recall_score(y_test1, y_pred_lgbm))

RandomForest:
Accuracy: 0.945054945054945
Recall: 0.9090909090909091
 
GradientBoosting:
Accuracy: 0.9340659340659341
Recall: 0.8863636363636364
 
XGBoost:
Accuracy: 0.9230769230769231
Recall: 0.8863636363636364
 
LightGBM:
Accuracy: 0.9560439560439561
Recall: 0.9318181818181818


In [ ]:
# 음성 모델 ensemble

ensemble_wav = VotingClassifier(estimators=[('rf', rf_rgs1), ('gb', gb_kf_rgs), ('xgb', xgb_gs), ('lgbm', lgb_kf_rgs)], voting='soft')
ensemble_wav.fit(X_train1, y_train1)
y_pred1 = ensemble_wav.predict(X_test1)

print("ensemble Recall:", recall_score(y_test1, y_pred1))
print("Accuracy:", accuracy_score(y_test1, y_pred1))

ensemble Recall: 0.9024390243902439
Accuracy: 0.9120879120879121


# 텍스트 데이터를 통한 보이스피싱 분류

In [2]:
# 일반대화 wav 데이터 불러오기 및 라벨링

os.chdir('/content/drive/MyDrive/1조 조별프로젝트/일반 보이스 합친 파일/유튭 합친 json')
text_dir = '/content/drive/MyDrive/1조 조별프로젝트/일반 보이스 합친 파일/유튭 합친 json'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass


d = pd.DataFrame(d)
normal= d
normal.rename(columns={0:'txt'},inplace=True)
normal['voice_fishing']=0
normal

,txt,voice_fishing
0,내가 나한테 잘해주면 돼요.혹시 여러분들 자식들가운데?지금 손자 손녀키운다고 먹고 ...,0
1,고맙습니다.한국방송통신대학교 경영학과에서 마케팅 분야를 담당하고 있고요.이반 학기소...,0
2,진동운동을 하게 되죠.따라서 원자를 알면 모든 것을 알 수 있다는 뜻이죠.양자역학은...,0
3,혹시 보령약국을 아시는 분이 계시죠?그러면 다음으로 상표권에 내용을 보도록 하겠습니...,0
4,우리 무사들 무시나오는데요.유붕자원방내 아니 불형 낳고 라 그러니까 멀리에서 친구가...,0
...,...,...
95,근데 이것도 행성한지 좀 됐는데 이걸 매일 매일 아직도 못 받아가지고 좀 확인이 가...,0
96,여기 빨간색화되는 것 중에 그냥 기본 심하면 또 되는 거네요 여기 인 사람들은 즐거...,0
97,그래서 해당 매일 들어가셔가지고 저희가 발송해 드린 인턴을 매일 보시면 아마 그 강...,0
98,이제 이게 아무래도 공공기관같은 경우는 블라인드 채용이 들어가서 경우가 많아서 블라...,0


In [3]:
# 보이스피싱 wav 데이터 불러오기 및 라벨링

os.chdir('/content/drive/MyDrive/1조 조별프로젝트/보이스피싱 음성 합친거/보이스피싱 json')
text_dir = '/content/drive/MyDrive/1조 조별프로젝트/보이스피싱 음성 합친거/보이스피싱 json'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass
d = pd.DataFrame(d)

voice= d
voice.rename(columns={0:'txt'},inplace=True)
voice['voice_fishing']=1

voice

,txt,voice_fishing
0,그리고 녹취 라고 해서 뭐 크게 별다른 건 없고 아까 제가 드렸던 질문을 그대로 녹...,1
1,지금 불법자금 불법게다가 섞여 있는 게 있는지 이렇게 진진의 파악을 하기 위해서 이...,1
2,본인명의로 하고 신화는 불법 통장이 발견되셨어요.미모가능 하신거죠 제가 사건 본 하...,1
3,혹시 그럼 최근 이 삼 년 사이에 개인정보가 유출이 될 만한 신분증이라든지 지갑여권...,1
4,중에 통장하나가 본인 명의로 개설돼 있는 제일의 대통장이 대포통장으로 같이 발견이 ...,1
...,...,...
77,따라서 따라서 물건을 받지 못한 피해자분들이 발생이 되었고 그 피해자 분들이 현재 ...,1
78,그렇기 때문에 저희가 몇 가지 확인을 해야 되는 부분이 있어서 연락을 드린건데 통합...,1
79,저희가 삼백명을 놓고 조사를 하는 중에 어떤 분들은 직접 통상을 양도판매한 사실이 ...,1
80,일단 이 두 통장은 범죄로 사용돼서 추가적인 피해자가 발생하지 않도록 본인 의사와 ...,1


In [4]:
# 일반대화와 보이스피싱 합치기
df = pd.concat([voice,normal],axis=0)
df.reset_index(drop=True,inplace=True)

In [5]:
X_train, X_val_test, y_train, y_val_test = train_test_split(df['txt'], df['voice_fishing'], test_size=0.5, random_state=46)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=46)

In [9]:
train_text=list(X_train)
label = y_train.values
val_text=list(X_val_test)
label2 = y_val_test.values
test_text=list(X_test)
label3 = y_test.values

In [10]:
# 데이터 수집 및 전처리
# (보이스피싱 및 일반 통화 데이터를 가져와 전처리 작업 수행)

# BERT 모델 불러오기
model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Fine-tuning
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 데이터를 토큰화하고 정수 인덱스로 변환
input_ids = []
labels = []

for text, label in zip(train_text, label):
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    input_ids.append(encoded_text)
    labels.append(label)


# 입력 시퀀스 패딩
max_seq_length = 128
input_ids = torch.tensor(pad_sequences(input_ids, maxlen=max_seq_length, padding='post'))
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# 모델 학습
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()
        batch_input_ids, batch_labels = batch
        outputs = model(input_ids=batch_input_ids, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    print(f"Epoch {epoch+1} average loss: {avg_loss}")

# 모델 평가
model.eval()
total_correct = 0
total_samples = 0

# 데이터를 토큰화하고 정수 인덱스로 변환
input_ids_test = []
labels_test = []

for text, label in zip(val_text, label2):
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    input_ids_test.append(encoded_text)
    labels_test.append(label)

# 입력 시퀀스 패딩
input_ids_test = torch.tensor(pad_sequences(input_ids_test, maxlen=max_seq_length, padding='post'))
labels_test = torch.tensor(labels_test)

# 테스트 데이터셋 생성
test_dataset = TensorDataset(input_ids_test, labels_test)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)



for batch in test_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(input_ids=batch_input_ids)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
    total_correct += (predicted_labels == batch_labels).sum().item()
    total_samples += len(batch_labels)

accuracy = total_correct / total_samples
print(f"Accuracy: {accuracy}")

# 추론
def predict(input_text):
    model.eval()
    with torch.no_grad():
        input_ids = torch.tensor([tokenizer.encode(input_text, add_special_tokens=True)])
        outputs = model(input_ids)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        if predicted_label == 0:
            print("일반 통화입니다.")
        else:
            print("보이스피싱입니다.")

# 테스트
input_text = "전화 요금이 이상하게 많이 나와서 확인하려고요."
predict(input_text)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (726 > 512). Running this sequence through the model will result in indexing errors



======== Epoch 1 / 20 ========
Epoch 1 average loss: 0.7006941437721252

======== Epoch 2 / 20 ========
Epoch 2 average loss: 0.6925298670927683

======== Epoch 3 / 20 ========
Epoch 3 average loss: 0.6648702025413513

======== Epoch 4 / 20 ========
Epoch 4 average loss: 0.6685220698515574

======== Epoch 5 / 20 ========
Epoch 5 average loss: 0.6555338402589163

======== Epoch 6 / 20 ========
Epoch 6 average loss: 0.6315845747788748

======== Epoch 7 / 20 ========
Epoch 7 average loss: 0.6049211323261261

======== Epoch 8 / 20 ========
Epoch 8 average loss: 0.5626950909694036

======== Epoch 9 / 20 ========
Epoch 9 average loss: 0.4355841775735219

======== Epoch 10 / 20 ========
Epoch 10 average loss: 0.3713847150405248

======== Epoch 11 / 20 ========
Epoch 11 average loss: 0.36705973744392395

======== Epoch 12 / 20 ========
Epoch 12 average loss: 0.33203261345624924

======== Epoch 13 / 20 ========
Epoch 13 average loss: 0.24867803355058035

======== Epoch 14 / 20 ========
Epoch 1

In [15]:
# 테스트 데이터를 토큰화하고 정수 인덱스로 변환
input_ids_test = []
labels_test = []

for text, label in zip(test_text, label3):
    encoded_text = tokenizer.encode(text, add_special_tokens=True)
    input_ids_test.append(encoded_text)
    labels_test.append(label)

# 입력 시퀀스 패딩
input_ids_test = torch.tensor(pad_sequences(input_ids_test, maxlen=max_seq_length, padding='post'))
labels_test = torch.tensor(labels_test)

# 테스트 데이터셋 생성
test_dataset = TensorDataset(input_ids_test, labels_test)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 추론 및 평가
model.eval()
total_correct = 0
total_samples = 0
tp = 0  # true positive
fn = 0  # false negative

for batch in test_dataloader:
    batch_input_ids, batch_labels = batch
    with torch.no_grad():
        outputs = model(input_ids=batch_input_ids)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
    total_correct += (predicted_labels == batch_labels).sum().item()
    total_samples += len(batch_labels)

    tp += ((predicted_labels == 1) & (batch_labels == 1)).sum().item()
    fn += ((predicted_labels == 0) & (batch_labels == 1)).sum().item()

accuracy = total_correct / total_samples
recall = tp / (tp + fn)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")

Accuracy: 0.9347826086956522
Recall: 0.9130434782608695


# 음성과 텍스트 을 모두 활용한 보이스피싱 분류

In [ ]:
# 음성과 텍스트를 모두 활용한 앙상블 모델

# 앙상블 모델 생성 및 예측
ensemble_model = VotingClassifier(estimators=[('bert', model), ('wav', ensemble_wav)], voting='soft')
ensemble_model.fit(predicted_labels, y_pred1)

# 앙상블 모델 평가
ensemble_predictions = ensemble_model.predict(predicted_labels)

print("ensemble Recall:", recall_score(ensemble_predictions, label3))
print("Accuracy:", accuracy_score(ensemble_predictions, label3))

ensemble Recall: 0.9318181818181818
Accuracy: 0.967032967032967
